In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
from utils import mnist_reader
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

def Linear(in_features, out_features, bias=True):

    m = nn.Linear(in_features, out_features, bias)
    nn.init.xavier_uniform_(m.weight)
    if bias:
        nn.init.constant_(m.bias, 0.0)
    return m

class myDataset(Dataset):
    def __init__(self,node_features,adj_features):
        self.node_features = node_features
        self.adj_features = adj_features
        self.max_size = self.node_features.shape[1]
        self.node_dim = self.node_features.shape[2]
        self.edge_dim = self.adj_features.shape[1]
        self.n_samples = self.node_features.shape[0]
    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        X = self.node_features[idx]
        adj = self.adj_features[idx]

        return torch.Tensor(X), torch.Tensor(adj)
    

In [2]:
#Single GCN Layer for Autoencoder
class GCN_layer(nn.Module):
    """
    Relation GCN layer. 
    """

    def __init__(self, in_features, out_features, edge_dim=3, aggregate='sum', dropout=0.0, use_relu=True, bias=False):
        '''
        parameters:
        in/out_features: embedding dimension of input/output
        edge_dim: dim of edge type
        aggregate: Type of aggregation to be used for pooling
        '''
        super(GCN_layer, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.edge_dim = edge_dim
        self.dropout = dropout
        self.aggregate = aggregate
        if use_relu:
            self.act = nn.ReLU()
        else:
            self.act = None

        self.weight = nn.Parameter(torch.FloatTensor(
            self.edge_dim, self.in_features, self.out_features)) 
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(
                self.edge_dim, self.out_features)) 
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.constant_(self.bias, 0.0)

    def forward(self, x, adj):
        '''
        parameters:
        x: (batch, N, NODE_FEATURES)
        adj: (batch, EDGE_FEATURES, N, N)
        
        returns:
        embedding with shape (batch, N, d)
        '''
        
        x = F.dropout(x, p=self.dropout, training=self.training)  # (batch, N, d)

        batch_size = x.size(0)

        #form layer output

        support = torch.einsum('bid, edh-> beih', x, self.weight)
        output = torch.einsum('...ij, ...jh-> ...ih', adj, support)  # (batch, EDGE_FEATURES, N, d)

        if self.bias is not None:
            output += self.bias
        if self.act is not None:
            output = self.act(output)  # (batch, EDGE_FEATURES, N, d)
        output = output.view(batch_size, self.edge_dim, x.size( 
            1), self.out_features)  # (batch, EDGE_FEATURES, N, d) 

        #Aggregates over each of the edge dimensions
        if self.aggregate == 'sum':
            # sum pooling #(batch, N, d)
            node_embedding = torch.sum(output, dim=1, keepdim=False)
        elif self.aggregate == 'max':
            # max pooling  #(batch, N, d)
            node_embedding = torch.max(output, dim=1, keepdim=False)
        elif self.aggregate == 'mean':
            # mean pooling #(batch, N, d)
            node_embedding = torch.mean(output, dim=1, keepdim=False)
        elif self.aggregate == 'none':
            node_embedding = output
        else:
            print('GCN aggregate error!')


        return node_embedding

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [9]:
#Autoencoder with GCN layers
class Autoencoder(nn.Module):
    def __init__(self, nfeat, nhid,nout=128,edge_dim =3, num_layers=5, dropout=0.0, normalization=False,gcn_bias=False):
        
        '''
        parameters:
        n_feat: number of node features
        n_hid: array of embedding sizes for each hidden layer
        n_out: embedding size for last layer
        '''
        super(Autoencoder, self).__init__()

        self.nfeat = nfeat
        self.nhid = nhid
        self.nout = nout
        self.edge_dim = edge_dim
        self.num_layers = num_layers

        self.dropout = dropout
        self.normalization = normalization

        self.emb = Linear(nfeat, nfeat, bias=False) 
        #self.bn_emb = nn.BatchNorm2d(8)

        self.gc1 = GCN_layer(
            nfeat, nhid[0], edge_dim=self.edge_dim, aggregate='sum', use_relu=True, dropout=self.dropout, bias = gcn_bias)
        if self.normalization:
            self.bn1 = nn.BatchNorm2d(nhid[0])

        self.gc2 = nn.ModuleList([GCN_layer(nhid[i], nhid[i+1], edge_dim=self.edge_dim, aggregate='sum',
                                                           use_relu=True, dropout=self.dropout, bias= gcn_bias)
                                  for i in range(self.num_layers-2)])
        if self.normalization:
            self.bn2 = nn.ModuleList([nn.BatchNorm2d(nhid[i+1]) for i in range(self.num_layers-2)])

        self.gc3 = GCN_layer(
            nhid[-1], nout, edge_dim=self.edge_dim, aggregate='none', use_relu=False, dropout=self.dropout, bias= gcn_bias)
        
        if self.normalization:
            self.bn3 = nn.BatchNorm2d(nout)
            
        self.activation = nn.Sigmoid()

    def forward(self, x, adj):
        '''
        :param x: (batch, N, d)
        :param adj: (batch, E, N, N)
        :return:
        '''
        # TODO: Add normalization for adacency matrix
        # embedding layer
        
        x = self.emb(x)
        #if self.normalization:
            #x = self.bn_emb(x.transpose(0, 3, 1, 2))
            #x = x.transpose(0, 2, 3, 1)

        # first GCN layer
        x = self.gc1(x, adj)
        #if self.normalization:
            #x = self.bn1(x.transpose(0, 3, 1, 2))
            #x = x.transpose(0, 2, 3, 1)

        # hidden GCN layer(s)
        for i in range(self.num_layers-2):
            x = self.gc2[i](x, adj)  # (#node, #class)
            #if self.normalization:
                #x = self.bn2[i](x.transpose(0, 3, 1, 2))
                #x = x.transpose(0, 2, 3, 1)

        # last GCN layer
        x = self.gc3(x, adj)  # (batch, N, d)
        #if self.normalization:
            #x = self.bn3(x.transpose(0, 3, 1, 2))
            #x = x.transpose(0, 2, 3, 1)

        #Reconstructs the adjacency matrix as x * x^T
        xt = torch.transpose(x,2,3)
        A = torch.einsum('bend, bedj-> benj', x, xt)
        
        return self.activation(A)

In [10]:
def train_epoch(model,criterion,optimizer,train_loader,device):
    model.train()
    total_loss = 0.0
    total = 0
    for X,adj in train_loader:
        optimizer.zero_grad()
        X = X.to(device)
        adj = adj.to(device)
        pred = model(X,adj)
        loss = criterion(pred,adj)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total += X.shape[0]
        
    return total_loss / total
        

In [14]:
num_nodes = 20
batch = 100
node_dim = 4
edge_dim = 12
nhid = [128,64,32,64]

x = torch.randn(batch,num_nodes,node_dim)
a = torch.randn(batch,edge_dim,num_nodes,num_nodes)

model = Autoencoder(nfeat = node_dim,nhid=nhid, edge_dim = edge_dim)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.MSELoss()
#Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = myDataset(x,a)
train_loader = DataLoader(dataset=train_set,batch_size=16,shuffle=True)

n_epochs = 10
for epoch in range(n_epochs):
    loss = train_epoch(model,criterion,optimizer,train_loader,device)
    print('Epoch',epoch,'loss:',loss)

Epoch 0 loss: 0.10659447431564331
Epoch 1 loss: 0.10648775935173034
Epoch 2 loss: 0.1066681170463562
Epoch 3 loss: 0.10653433203697205
Epoch 4 loss: 0.10654185771942139
Epoch 5 loss: 0.10686003684997558
Epoch 6 loss: 0.10682871699333191
Epoch 7 loss: 0.10664951920509338
Epoch 8 loss: 0.10664633512496949
Epoch 9 loss: 0.10675543785095215
